# Improved Data Visualization and Model Training for NAVAR_TCN_EPISTEMIC

In [1]:
import sys

# Add the project directory to the sys.path
sys.path.append("../")

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import trange
import seaborn as sns

from src.models.navar.navar_epistemic import NAVAR_TCN_E, loss_navar_e
from scripts.experiments.remember_noise_test_epistemic import generate_random_data, train_model, post_training_analysis_navar
from src.utils.pytorch import count_parameters
from src.utils.visualisations import smooth_line

C:\Users\mauri\anaconda3\envs\causal-discovery\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'src.models.navar'

## Data Preparation

In [ ]:
x_train, y_train, true_function_train = generate_random_data(sequence_length=2500)

if torch.cuda.is_available():
    x_train, y_train, true_function_train = x_train.cuda(), y_train.cuda(), true_function_train.cuda()

plt.plot(x_train[0, 1, :100].cpu())

## Model Training

In [ ]:
model = NAVAR_TCN_E(n_variables=3, hidden_dim=16, kernel_size=2, n_blocks=1, n_layers_per_block=2, dropout=0.0)
print(model)

if torch.cuda.is_available():
    model = model.cuda()

print('n_parameters per var:', count_parameters(model) // 3)
print('receptive field:', model.receptive_field)

model, losses = train_model(model, x_train, y_train, loss_navar_e, epochs=1000, lr=2e-3, coeff_start=-4, coeff_end=-1)

In [ ]:
plt.plot(losses[100:])

## Data Visualization

In [ ]:
def plot_matrix():
    result = post_training_analysis_navar(model, x_train, y_train, true_function_train, from_var=0, to_var=0)
    sns.set(rc={'figure.figsize':(3,2.5)})
    sns.heatmap(result[7])
    plt.show()
    sns.set(rc={'figure.figsize':(3,2.5)})
    sns.heatmap(np.sqrt(1/result[8]**2))
    plt.show()
plot_matrix()

plot empistemic uncertainty


In [ ]:
def plot_relationships(from_var, to_var):
    x, y, true_function, gamma, v, aleatoric, epistemic, causal_matrix, confidence_matrix, \
    gamma_prime, aleatoric_prime, epistemic_prime = post_training_analysis_navar(model, x_train, y_train, true_function_train, from_var=from_var, to_var=to_var)
    
    gamma_smooth = smooth_line(gamma, 3.0)
    aleatoric_smooth = smooth_line(aleatoric, 3.0)
    epistemic_smooth = smooth_line(epistemic, 3.0)
    
    fig, ax = plt.subplots(figsize=(5, 3))

    # epistemic
    plt.fill_between(x, gamma_smooth - 0.4 * epistemic_smooth, gamma_smooth + 0.4 * epistemic_smooth, color='green', alpha=0.1, label='Epistemic')
    # aleatoric
    plt.plot(x, gamma_smooth - 2 * aleatoric_smooth, color='black', alpha=0.5, label='Aleatoric')
    plt.plot(x, gamma_smooth + 2 * aleatoric_smooth, color='black', alpha=0.5)
    # true function
    #plt.plot(x, true_function, label='True Function', color='blue', alpha=1.0)
    # train data
    plt.scatter(x, y, color='blue', marker='o', s=1, label='Train data', alpha=0.2)
    # prediction
    plt.plot(x, gamma_smooth, label='Prediction', color='red', alpha=1.0)
    
    plt.xlim([-5, 5])
    plt.ylim([-8, 8])
    plt.xlabel(f"X{from_var}(t-1)")
    plt.ylabel(f"X{to_var}(t)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"C:/Users/mauri/Desktop/thesis_img/uncertainty_relationship-{from_var}-{to_var}.svg", format='svg')

plot time series

In [ ]:
for i in range(3):
    for j in range(3):
        plot_relationships(j, i)

In [ ]:
def plot_timeseries(uncertainty_type):
    with torch.no_grad():
        gamma, v, aleatoric_var, aleatoric_log_var, epistemic_var, gamma_prime, v_prime, aleatoric_var_prime, epistemic_var_prime = model(x_train)
    
    a, b = 100, 300
    x_ticks = np.arange(a, b, 1)

    plt.figure(figsize=(12, 6))
    for sample_channel in range(3):
        ax = plt.subplot(3, 1, sample_channel + 1)  # 3 rows, 1 column, current plot
    
        plt.plot(x_ticks, y_train[0, sample_channel, a:b].cpu().numpy(),
                 label="Original Data", alpha=1.0, zorder=0, color='coral')
    
        plt.plot(x_ticks, true_function_train[0, sample_channel, a:b].cpu().numpy(),
                 label="True Contributions", alpha=0.4, zorder=1, color='red')


        gamma1 = gamma_prime[0, sample_channel, a:b].cpu().numpy()
        plt.plot(x_ticks, gamma1,
                 label="Model Prediction (μ)", linestyle='dashed', alpha=0.9, zorder=1, color='royalblue')

        if uncertainty_type == 'aleatoric':
            std1 = torch.sqrt(aleatoric_var_prime[0, sample_channel, a:b]).cpu().numpy()
            # Plotting standard deviations
            plt.fill_between(x_ticks,
                             gamma1 - 2 * std1,
                             gamma1 + 2 * std1,
                             color='green', alpha=0.3, zorder=-2)
        
            # Plotting standard deviations
            plt.fill_between(x_ticks,
                             gamma1 - std1,
                             gamma1 + std1,
                             color='green', alpha=0.3, label="Aleatoric Uncertainty (2σ)", zorder=-1)
            ax.set_ylim([-2.5, 2.5])
        else:
            std1 = torch.sqrt(epistemic_var_prime[0, sample_channel, a:b]).cpu().numpy()
            # Plotting standard deviations lightsteelblue
            plt.fill_between(x_ticks,
                             gamma1 - 0.4 * std1,
                             gamma1 + 0.4 * std1,
                             color='green', alpha=0.3, zorder=-2)
        
            # Plotting standard deviations
            plt.fill_between(x_ticks,
                             gamma1 - 0.2 *std1,
                             gamma1 + 0.2 * std1,
                             color='green', alpha=0.3, label="Epistemic Uncertainty (2σ)", zorder=-1)
            ax.set_ylim([-3, 3])
        
        if sample_channel == 0:
            plt.legend()
    
        if sample_channel != 2:  # if it's not the last plot, hide the x-axis
            plt.setp(ax.get_xticklabels(), visible=False)
    
        #plt.setp(ax.get_yticklabels(), visible=False)
        ax.set_ylabel(f"Var {sample_channel + 1}")
    
        ax.grid(False)
    plt.tight_layout()  # to prevent overlap of titles and labels
    plt.show()


In [ ]:
plot_timeseries('aleatoric')

In [ ]:
plot_timeseries('epistemic')

plot paramters

In [ ]:
def plot_params(from_var, to_var):
    x, y, true_function, gamma, v, aleatoric, epistemic, causal_matrix, confidence_matrix, \
    gamma_prime, aleatoric_prime, epistemic_prime = post_training_analysis_navar(model, x_train, y_train, true_function_train, from_var=from_var, to_var=to_var)
    
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 4))
    for i, (d, name) in enumerate([(gamma, 'mu'), (v, 'v'), (aleatoric, 'aleatoric'), (epistemic, 'epistemic')]):
        ax = axes[0, i]
        #k = len(d)
        #plt.plot(d[int(0.30 * len(d)): int(0.70 * len(d))])
        ax.plot(x, d)
        ax.set_title(name)
        
        ax = axes[1, i]
        ax.plot(np.log(d))
        ax.set_title(f"log({name})")
    plt.show()

In [ ]:
plot_params(0, 1)